<a href="https://colab.research.google.com/github/cyss93/CP1_team8/blob/%EB%B0%95%EC%A3%BC%EC%98%81/Olist_data_EDA_220629.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from haversine import haversine

In [4]:
# marketing funnel
df_closed = pd.read_csv('/content/drive/MyDrive/olist/olist_closed_deals_dataset.csv') #
df_qualified = pd.read_csv('/content/drive/MyDrive/olist/olist_marketing_qualified_leads_dataset.csv') #

# customers
df_items = pd.read_csv('/content/drive/MyDrive/olist/olist_order_items_dataset.csv') #
df_payments = pd.read_csv('/content/drive/MyDrive/olist/olist_order_payments_dataset.csv')
df_orders = pd.read_csv('/content/drive/MyDrive/olist/olist_orders_dataset.csv')

df_cus = pd.read_csv('/content/drive/MyDrive/olist/olist_customers_dataset.csv')
df_geo = pd.read_csv('/content/drive/MyDrive/olist/olist_geolocation_dataset.csv')
df_products = pd.read_csv('/content/drive/MyDrive/olist/olist_products_dataset.csv')
df_sellers = pd.read_csv('/content/drive/MyDrive/olist/olist_sellers_dataset.csv') #

df_translate = pd.read_csv('/content/drive/MyDrive/olist/product_category_name_translation.csv')

In [5]:
# marketing funnel 데이터 병합
df_funnel = df_qualified.merge(df_closed, on='mql_id', how='inner')

# df_funnel과 df_sellers 병합
df_funnel_sellers = df_funnel.merge(df_sellers, on='seller_id', how='inner')

# df_items 데이터 셋과 합치기
df_1 = df_funnel_sellers.merge(df_items, on='seller_id', how='inner')

# df_orders 데이터 셋과 합치기
df_2 = df_1.merge(df_orders, on='order_id', how='inner')

# df_orders 데이터 셋과 합치기
df_3 = df_2.merge(df_cus, on='customer_id', how='inner')

In [6]:
# 판매자, 구매자 위치 정보 합치기

df_geo_seller = df_geo.copy()
subset = df_geo_seller[['geolocation_lat', 'geolocation_lng']]
df_geo_seller['seller_loc'] = [tuple(x) for x in subset.to_numpy()]
df_geo_seller.drop(['geolocation_lat','geolocation_lng','geolocation_city',	'geolocation_state'], axis =1, inplace=True)
df_geo_seller.rename(columns = {'geolocation_zip_code_prefix':'seller_zip_code_prefix'},inplace=True)
df_geo_seller.drop_duplicates(['seller_zip_code_prefix'], keep = 'first', inplace=True)
df_geo_cus = df_geo_seller.copy()
df_geo_cus.rename(columns = {'seller_zip_code_prefix':'customer_zip_code_prefix','seller_loc':'cus_loc'},inplace=True)

df_4 = df_3.merge(df_geo_seller, on='seller_zip_code_prefix', how='inner')
df_5 = df_4.merge(df_geo_cus, on='customer_zip_code_prefix', how='inner')

In [7]:
# 위치 정보를 통한 거리 계산

subset2 = df_5[['seller_loc', 'cus_loc']]
df_5['distance']=[haversine(x,y) for x,y in subset2.to_numpy()]
df_5

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,...,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_loc,cus_loc,distance
0,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,2018-06-22 15:23:00,2018-06-26 20:06:25,2018-07-17 00:00:00,6cd62cc4c0a89fb414ae292c6da98efc,13733,mococa,SP,"(-19.939688821541896, -43.97533764669048)","(-21.461800979445574, -47.00077905378499)",357.306376
1,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,2018-07-25 15:09:00,2018-07-27 18:27:42,2018-08-07 00:00:00,7eb01c1ef439cd15ed3a4cc601d15dd8,14020,ribeirao preto,SP,"(-19.939688821541896, -43.97533764669048)","(-21.210104711049134, -47.80139005429387)",422.590461
2,0b97be8b4b408a3a0780a32e0a143a7f,2018-01-22,b76ef37428e6799c421989521c0e5077,unknown,7d13fca15225358621be4086e1eb0964,56bf83c4bb35763a51c2baab501b4c67,9ae085775a198122c5586fa830ff7f2b,2018-02-08 17:20:14,watches,online_big,...,2018-07-24 14:36:00,2018-07-25 17:03:26,2018-08-08 00:00:00,a9967806034d2d782bbcd181db432ee2,14020,ribeirao preto,SP,"(-21.17121748495008, -47.81941847157649)","(-21.210104711049134, -47.80139005429387)",4.710751
3,384008daaa0f14b58912d8648101fdd0,2018-02-27,ce1a65abd0973638f1c887a6efcfa82d,organic_search,8a432f4e5b471f8da497d7dc517666e2,9e4d1098a3b0f5da39b0bc48f9876645,c638112b43f1d1b86dcabb0da720c901,2018-03-12 17:30:06,bed_bath_table,online_big,...,2018-08-27 14:06:00,2018-08-30 20:39:51,2018-09-05 00:00:00,3f903ae26d5419e71b3eae9201f54c85,14020,ribeirao preto,SP,"(-22.133927157857254, -51.36875322173029)","(-21.210104711049134, -47.80139005429387)",382.665308
4,2f68c2e0cd8964d59dbbcfa48196b499,2018-03-05,88740e65d5d6b056e0cda098e1ea6313,social,4c1c7281388a33dd06daac44f9fadbd1,9e4d1098a3b0f5da39b0bc48f9876645,4ef15afb4b2723d8f3d81e51ec7afefe,2018-03-08 18:57:56,stationery,online_medium,...,2018-06-22 12:20:00,2018-06-25 21:33:14,2018-07-12 00:00:00,942507885a441e329ab26f393af451ba,14020,ribeirao preto,SP,"(-22.193082052455427, -49.948926747706174)","(-21.210104711049134, -47.80139005429387)",247.327339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,98826328c3a60215e53b342560084571,2018-04-04,73f31a40697cc90a86c1273563ac230e,other,f46490624488d3ff7ce78613913a7711,9d12ef1a7eca3ec58c545c678af7869c,068066e24f0c643eb1d089c7dd20cd73,2018-05-16 19:51:05,health_beauty,online_big,...,2018-06-21 14:41:00,2018-07-14 18:23:44,2018-07-24 00:00:00,51e12be240f7064232b983c4218f67f3,57075,maceio,AL,"(-23.444148887039063, -46.50655197737896)","(-9.550947098257252, -35.78991513946058)",1919.286173
5026,676207ddcc8087437135a4b461fc07cb,2018-04-08,58326e62183c14b0c03085c33b9fdc44,social,c70a353f02429c00775a46a75fb787da,370c9f455f93a9a96cbe9bea48e70033,4ef15afb4b2723d8f3d81e51ec7afefe,2018-04-20 12:27:44,health_beauty,online_medium,...,2018-08-09 13:37:00,2018-08-16 15:04:06,2018-09-06 00:00:00,c148e9b2cf4a2b58b7edefc55863c818,76829,porto velho,RO,"(-23.57714001165677, -46.48276229003407)","(-8.75950516543655, -63.829139181856405)",2473.976692
5027,676207ddcc8087437135a4b461fc07cb,2018-04-08,58326e62183c14b0c03085c33b9fdc44,social,c70a353f02429c00775a46a75fb787da,370c9f455f93a9a96cbe9bea48e70033,4ef15afb4b2723d8f3d81e51ec7afefe,2018-04-20 12:27:44,health_beauty,online_medium,...,2018-08-09 15:15:00,2018-08-10 20:48:30,2018-08-17 00:00:00,c2d79e2aec1562aee14ac1e8abee4c17,3057,sao paulo,SP,"(-23.57714001165677, -46.48276229003407)","(-23.54257772745109, -46.59338016847479)",11.911855
5028,676207ddcc80874

In [8]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5030 entries, 0 to 5029
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   mql_id                         5030 non-null   object 
 1   first_contact_date             5030 non-null   object 
 2   landing_page_id                5030 non-null   object 
 3   origin                         5019 non-null   object 
 4   seller_id                      5030 non-null   object 
 5   sdr_id                         5030 non-null   object 
 6   sr_id                          5030 non-null   object 
 7   won_date                       5030 non-null   object 
 8   business_segment               5030 non-null   object 
 9   lead_type                      4982 non-null   object 
 10  lead_behaviour_profile         3292 non-null   object 
 11  has_company                    28 non-null     object 
 12  has_gtin                       28 non-null     o

In [9]:
df_5['order_status'].value_counts()

delivered    4971
shipped        42
invoiced        9
canceled        8
Name: order_status, dtype: int64

In [10]:
# 특성 삭제 (1차)
remove = ['mql_id','sdr_id','sr_id','lead_behaviour_profile','has_company','has_gtin','average_stock','declared_product_catalog_size',
          'declared_monthly_revenue','seller_zip_code_prefix','seller_city','shipping_limit_date','customer_id','order_status',
          'order_purchase_timestamp','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date',
          'customer_unique_id','customer_zip_code_prefix','customer_city','customer_state','seller_loc','cus_loc']

"""
'mql_id' : 'seller_id'와 중복
'sdr_id','sr_id' : '판매자들의 담당자' 데이터
'lead_behaviour_profile' : 브라질식 mbti
'has_company','has_gtin','average_stock','declared_product_catalog_size' : 대부분이 결측치
'declared_monthly_revenue' : 예상 수익. 대부분이 0
'seller_zip_code_prefix' : 위치 정보 코드. 도시나 주 데이터와 내용상 중복
※주의※ 'seller_city' : 판매자 도시. 판매자 고유값이 379명이고 도시가 166개, 주가 13개라 다수가 묶이기 쉬운 주 데이터 선택. 분석상 필요시 교체할 것
※주의※ 'shipping_limit_date' : 물류 배송 제한 날짜? 해석이 어려움. 이후 의미 있는 데이터라 생각 되면 추가
'customer_id' : 주문자 데이터
'order_status' : 주문 상태. 대부분이 delivered
'order_purchase_timestamp','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date' : 'order_approved_at'을 포함한 5가지 시계열 데이터 중 실제 결제가 이루어진 시각을 수익 발생 시각으로 채택
'customer_unique_id','customer_zip_code_prefix','customer_city','customer_state' : 주문자 데이터
'seller_loc','cus_loc' : 거리 계산을 위해 만들었던 데이터
"""

In [11]:
df_6 = df_5.drop(remove, axis =1)

In [12]:
df_6

,first_contact_date,landing_page_id,origin,seller_id,won_date,business_segment,lead_type,business_type,seller_state,order_id,order_item_id,product_id,price,freight_value,order_approved_at,distance
0,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,2018-02-26 19:58:54,pet,online_medium,reseller,MG,1efcc6ba75b519535a510c53153a9256,1,e94df75b2e9a8b056be0c144a731f55c,94.20,18.76,2018-06-20 22:20:21,357.306376
1,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,2018-02-26 19:58:54,pet,online_medium,reseller,MG,d8d281c1a1398a1e835fdc4adfb81e2a,1,9e0873e9ac04b12f282b81af5daebd2e,664.00,38.41,2018-07-24 10:31:11,422.590461
2,2018-01-22,b76ef37428e6799c421989521c0e5077,unknown,7d13fca15225358621be4086e1eb0964,2018-02-08 17:20:14,watches,online_big,reseller,SP,9dfd8df84d4f07cc8da0f2fa0fe96075,1,ba74c6b75d2ad7503175809688d5a03c,178.99,8.99,2018-07-20 04:43:20,4.710751
3,2018-02-27,ce1a65abd0973638f1c887a6efcfa82d,organic_search,8a432f4e5b471f8da497d7dc517666e2,2018-03-12 17:30:06,bed_bath_table,online_big,reseller,SP,931ca162115da049cd8d7872b3667163,1,e8f487b9cb03fdb7daf2505822570853,68.00,14.06,2018-08-25 08:50:12,382.665308
4,2018-03-05,88740e65d5d6b056e0cda098e1ea6313,social,4c1c7281388a33dd06daac44f9fadbd1,2018-03-08 18:57:56,stationery,online_medium,reseller,SP,e8f4435c1db6d7bf9e191fd9bd6a6682,1,38f2d4b26820d63dc2675d6fbfafe1bb,32.89,12.89,2018-06-20 19:58:06,247.327339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,2018-04-04,73f31a40697cc90a86c1273563ac230e,other,f46490624488d3ff7ce78613913a7711,2018-05-16 19:51:05,health_beauty,online_big,reseller,SP,f9b709a84c50f277996aa9f181738b73,1,ef7f13c7694be202c48017b71801815c,49.90,24.70,2018-06-21 11:58:50,1919.286173
5026,2018-04-08,58326e62183c14b0c03085c33b9fdc44,social,c70a353f02429c00775a46a75fb787da,2018-04-20 12:27:44,health_beauty,online_medium,reseller,SP,4471fc5ecfbb3b7fec1044fc6e6f6851,1,1b34de23397f9ce6cd0f7afa27b60eac,38.50,22.99,2018-08-08 18:15:21,2473.976692
5027,2018-04-08,58326e62183c14b0c03085c33b9fdc44,social,c70a353f02429c00775a46a75fb787da,2018-04-20 12:27:44,health_beauty,online_medium,reseller,SP,56aa3b6b2c30314ee80914fea9543573,1,059edbecfc72eba55fbd079a25c4ab6b,35.00,9.00,2018-08-08 20:04:53,11.911855
5028,2018-04-08,58326e62183c14b0c03085c33b9fdc44,social,c70a353f02429c00775a46a75fb787da,2018-04-20 12:27:44,health_beauty,online_medium,reseller,SP,935cbf337f50d03fc7ecfb5a601dc7fc,1,1b34de23397f9ce6cd0f7afa27b60eac,38.50,19.18,2018-07-25 08:25:14,2121.435422


In [13]:
df_6['seller_profit']=df_6['price']-df_6['freight_value']

In [14]:
df_6.columns

Index(['first_contact_date', 'landing_page_id', 'origin', 'seller_id',
       'won_date', 'business_segment', 'lead_type', 'business_type',
       'seller_state', 'order_id', 'order_item_id', 'product_id', 'price',
       'freight_value', 'order_approved_at', 'distance', 'seller_profit'],
      dtype='object')

In [15]:
order_seller_data = df_6[['order_id','product_id','order_item_id','price','freight_value','seller_profit','distance','order_approved_at',
                          'seller_id','origin','landing_page_id','first_contact_date','won_date',
                          'business_type','lead_type','business_segment','seller_state']]

In [16]:
order_seller_data['order_approved_at'] = pd.to_datetime(order_seller_data['order_approved_at'])
order_seller_data['first_contact_date'] = pd.to_datetime(order_seller_data['first_contact_date'])
order_seller_data['won_date'] = pd.to_datetime(order_seller_data['won_date'])

In [17]:
# 주문 - 판매자 데이터
order_seller_data

,order_id,product_id,order_item_id,price,freight_value,seller_profit,distance,order_approved_at,seller_id,origin,landing_page_id,first_contact_date,won_date,business_type,lead_type,business_segment,seller_state
0,1efcc6ba75b519535a510c53153a9256,e94df75b2e9a8b056be0c144a731f55c,1,94.20,18.76,75.44,357.306376,2018-06-20 22:20:21,2c43fb513632d29b3b58df74816f1b06,organic_search,b48ec5f3b04e9068441002a19df93c6c,2018-02-21,2018-02-26 19:58:54,reseller,online_medium,pet,MG
1,d8d281c1a1398a1e835fdc4adfb81e2a,9e0873e9ac04b12f282b81af5daebd2e,1,664.00,38.41,625.59,422.590461,2018-07-24 10:31:11,2c43fb513632d29b3b58df74816f1b06,organic_search,b48ec5f3b04e9068441002a19df93c6c,2018-02-21,2018-02-26 19:58:54,reseller,online_medium,pet,MG
2,9dfd8df84d4f07cc8da0f2fa0fe96075,ba74c6b75d2ad7503175809688d5a03c,1,178.99,8.99,170.00,4.710751,2018-07-20 04:43:20,7d13fca15225358621be4086e1eb0964,unknown,b76ef37428e6799c421989521c0e5077,2018-01-22,2018-02-08 17:20:14,reseller,online_big,watches,SP
3,931ca162115da049cd8d7872b3667163,e8f487b9cb03fdb7daf2505822570853,1,68.00,14.06,53.94,382.665308,2018-08-25 08:50:12,8a432f4e5b471f8da497d7dc517666e2,organic_search,ce1a65abd0973638f1c887a6efcfa82d,2018-02-27,2018-03-12 17:30:06,reseller,online_big,bed_bath_table,SP
4,e8f4435c1db6d7bf9e191fd9bd6a6682,38f2d4b26820d63dc2675d6fbfafe1bb,1,32.89,12.89,20.00,247.327339,2018-06-20 19:58:06,4c1c7281388a33dd06daac44f9fadbd1,social,88740e65d5d6b056e0cda098e1ea6313,2018-03-05,2018-03-08 18:57:56,reseller,online_medium,stationery,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,f9b709a84c50f277996aa9f181738b73,ef7f13c7694be202c48017b71801815c,1,49.90,24.70,25.20,1919.286173,2018-06-21 11:58:50,f46490624488d3ff7ce78613913a7711,other,73f31a40697cc90a86c1273563ac230e,2018-04-04,2018-05-16 19:51:05,reseller,online_big,health_beauty,SP
5026,4471fc5ecfbb3b7fec1044fc6e6f6851,1b34de23397f9ce6cd0f7afa27b60eac,1,38.50,22.99,15.51,2473.976692,2018-08-08 18:15:21,c70a353f02429c00775a46a75fb787da,social,58326e62183c14b0c03085c33b9fdc44,2018-04-08,2018-04-20 12:27:44,reseller,online_medium,health_beauty,SP
5027,56aa3b6b2c30314ee80914fea9543573,059edbecfc72eba55fbd079a25c4ab6b,1,35.00,9.00,26.00,11.911855,2018-08-08 20:04:53,c70a353f02429c00775a46a75fb787da,social,58326e62183c14b0c03085c33b9fdc44,2018-04-08,2018-04-20 12:27:44,reseller,online_medium,health_beauty,SP
5028,935cbf337f50d03fc7ecfb5a601dc7fc,1b34de23397f9ce6cd0f7afa27b60eac,1,38.50,19.18,19.32,2121.435422,2018-07-25 08:25:14,c70a353f02429c00775a46a75fb787da,social,58326e62183c14b0c03085c33b9fdc44,2018-04-08,2018-04-20 12:27:44,reseller,online_medium,health_beauty,SP


In [36]:
order_data = order_seller_data[['order_id','product_id','order_item_id','price','freight_value','seller_profit','distance','order_approved_at','seller_id']]

In [18]:
seller_data= [['seller_id','origin','landing_page_id','first_contact_date','won_date','business_type','lead_type','business_segment','seller_state']]

In [19]:
seller_data.drop_duplicates(inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [20]:
# 판매자 고유 데이터
seller_data

,seller_id,origin,landing_page_id,first_contact_date,won_date,business_type,lead_type,business_segment,seller_state
0,2c43fb513632d29b3b58df74816f1b06,organic_search,b48ec5f3b04e9068441002a19df93c6c,2018-02-21,2018-02-26 19:58:54,reseller,online_medium,pet,MG
2,7d13fca15225358621be4086e1eb0964,unknown,b76ef37428e6799c421989521c0e5077,2018-01-22,2018-02-08 17:20:14,reseller,online_big,watches,SP
3,8a432f4e5b471f8da497d7dc517666e2,organic_search,ce1a65abd0973638f1c887a6efcfa82d,2018-02-27,2018-03-12 17:30:06,reseller,online_big,bed_bath_table,SP
4,4c1c7281388a33dd06daac44f9fadbd1,social,88740e65d5d6b056e0cda098e1ea6313,2018-03-05,2018-03-08 18:57:56,reseller,online_medium,stationery,SP
6,1d953075c2f0dd990bacf27b83b330f1,organic_search,a7982125ff7aa3b2054c6e44f9d28522,2018-05-31,2018-06-20 12:24:10,reseller,online_medium,computers,MG
...,...,...,...,...,...,...,...,...,...
4792,88cd3681e0a2ba85e190f7d817629c4c,paid_search,b48ec5f3b04e9068441002a19df93c6c,2017-11-14,2018-07-09 18:00:57,reseller,online_big,car_accessories,SC
4845,e116c7455dd26a4d8c3e92532583905f,unknown,b76ef37428e6799c421989521c0e5077,2018-01-12,2018-01-22 02:00:00,reseller,online_big,health_beauty,SP
4879,8fb67a334bacec338885b4e9e1708073,paid_search,40dec9f3d5259a3d2dbcdab2114fae47,2018-04-24,2018-04-27 12:18:34,manufacturer,online_medium,home_decor,PR
4915,0cab2da43793a6f3c5ed8514c5f54627,NaN,6b6c0934205e87702519d1cce24055f9,2018-03-12,2018-04-05 16:29:23,reseller,offline,bags_backpacks,PR


In [21]:
# 판매자 등록까지 걸린 시간
seller_data['date_for_registration'] = (seller_data['won_date']-seller_data['first_contact_date']).dt.days

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
# 판매자 - 판매 기간 데이터 (등록일 ~ 마지막 판매 기간)
df_sale_days = pd.DataFrame({'sale_days' :[(order_seller_data.groupby('seller_id')['order_approved_at'].max() - order_seller_data.groupby('seller_id')['won_date'].max()).dt.days][0]})
df_sale_days.reset_index(inplace = True)
df_sale_days

,seller_id,sale_days
0,01266d4c46afa519678d16a8b683d325,94
1,01fd077212124329bac32490e8ef80d9,149
2,02f623a8eb246f3c5f7c2f96462654e6,126
3,03a2a6dcbe45d639c4dc2a0d24769c40,187
4,044668ccd5316b12a7cf0d54a156e3e9,201
...,...,...
374,fd435faa3c0422b60440ea3480d0e77c,24
375,fe87f472055fbcf1d7e691c00b1560dc,209
376,ffa6adafb71b807dc13159e26431354c,174
377,ffad1e7127fb622cb64a900751590acd,161


In [23]:
# 판매자 - 판매 수익 데이터
df_total_profit = pd.DataFrame({'total_profit' :[order_seller_data.groupby('seller_id')['seller_profit'].sum()][0]})
df_total_profit.reset_index(inplace = True)
df_total_profit

,seller_id,total_profit
0,01266d4c46afa519678d16a8b683d325,43.02
1,01fd077212124329bac32490e8ef80d9,632.64
2,02f623a8eb246f3c5f7c2f96462654e6,5548.37
3,03a2a6dcbe45d639c4dc2a0d24769c40,906.16
4,044668ccd5316b12a7cf0d54a156e3e9,84.25
...,...,...
374,fd435faa3c0422b60440ea3480d0e77c,615.13
375,fe87f472055fbcf1d7e691c00b1560dc,955.64
376,ffa6adafb71b807dc13159e26431354c,2132.82
377,ffad1e7127fb622cb64a900751590acd,677.77


In [24]:
# 판매자 - 평균 판매 거리 데이터
df_mean_distance = pd.DataFrame({'mean_distance' :[order_seller_data.groupby('seller_id')['distance'].mean()][0]})
df_mean_distance.reset_index(inplace = True)
df_mean_distance

,seller_id,mean_distance
0,01266d4c46afa519678d16a8b683d325,366.188948
1,01fd077212124329bac32490e8ef80d9,464.735110
2,02f623a8eb246f3c5f7c2f96462654e6,471.067035
3,03a2a6dcbe45d639c4dc2a0d24769c40,594.522556
4,044668ccd5316b12a7cf0d54a156e3e9,569.389775
...,...,...
374,fd435faa3c0422b60440ea3480d0e77c,1971.258549
375,fe87f472055fbcf1d7e691c00b1560dc,181.081762
376,ffa6adafb71b807dc13159e26431354c,790.681428
377,ffad1e7127fb622cb64a900751590acd,404.112355


In [25]:
# 판매자 - 이탈 여부 데이터 (최근 60 일간 판매가 없을 시 이탈로 간주)
df_seller_status = pd.DataFrame({'last' :[(order_seller_data['order_approved_at'].max()-order_seller_data.groupby('seller_id')['order_approved_at'].max()).dt.days][0]})
df_seller_status.reset_index(inplace = True)
df_seller_status['seller_status']=["leave" if x > 60 else "maintain" for x in df_seller_status['last']]
df_seller_status.drop(['last'], axis=1, inplace=True)
df_seller_status

,seller_id,seller_status
0,01266d4c46afa519678d16a8b683d325,maintain
1,01fd077212124329bac32490e8ef80d9,maintain
2,02f623a8eb246f3c5f7c2f96462654e6,maintain
3,03a2a6dcbe45d639c4dc2a0d24769c40,maintain
4,044668ccd5316b12a7cf0d54a156e3e9,maintain
...,...,...
374,fd435faa3c0422b60440ea3480d0e77c,maintain
375,fe87f472055fbcf1d7e691c00b1560dc,maintain
376,ffa6adafb71b807dc13159e26431354c,maintain
377,ffad1e7127fb622cb64a900751590acd,maintain


In [26]:
# 데이터 합치기
seller_data = seller_data.merge(df_sale_days, on='seller_id', how='inner')
seller_data = seller_data.merge(df_total_profit, on='seller_id', how='inner')
seller_data = seller_data.merge(df_mean_distance, on='seller_id', how='inner')
seller_data = seller_data.merge(df_seller_status, on='seller_id', how='inner')

# LTV 계산
seller_data['LTV'] = seller_data['total_profit'] / seller_data['sale_days']

In [28]:
seller_data.columns

Index(['seller_id', 'origin', 'landing_page_id', 'first_contact_date',
       'won_date', 'business_type', 'lead_type', 'business_segment',
       'seller_state', 'date_for_registration', 'sale_days', 'total_profit',
       'mean_distance', 'seller_status', 'LTV'],
      dtype='object')

In [31]:
# 특성 삭제 (2차)
remove2 = ['first_contact_date', 'won_date']

# 'first_contact_date', 'won_date' : 'date_for_registration' 특성을 만들기 위한 특성

In [32]:
seller_data.drop(remove2, axis=1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
seller_data= seller_data[['seller_id', 'origin', 'landing_page_id','date_for_registration','business_type', 'lead_type', 'business_segment',
                          'seller_state','mean_distance','sale_days', 'total_profit','LTV','seller_status']]

In [34]:
seller_data

,seller_id,origin,landing_page_id,date_for_registration,business_type,lead_type,business_segment,seller_state,mean_distance,sale_days,total_profit,LTV,seller_status
0,2c43fb513632d29b3b58df74816f1b06,organic_search,b48ec5f3b04e9068441002a19df93c6c,5,reseller,online_medium,pet,MG,388.692364,147,785.83,5.345782,maintain
1,7d13fca15225358621be4086e1eb0964,unknown,b76ef37428e6799c421989521c0e5077,17,reseller,online_big,watches,SP,621.754948,197,104805.89,532.009594,maintain
2,8a432f4e5b471f8da497d7dc517666e2,organic_search,ce1a65abd0973638f1c887a6efcfa82d,13,reseller,online_big,bed_bath_table,SP,666.394410,168,5589.89,33.273155,maintain
3,4c1c7281388a33dd06daac44f9fadbd1,social,88740e65d5d6b056e0cda098e1ea6313,3,reseller,online_medium,stationery,SP,491.260670,104,106.73,1.026250,leave
4,1d953075c2f0dd990bacf27b83b330f1,organic_search,a7982125ff7aa3b2054c6e44f9d28522,20,reseller,online_medium,computers,MG,1334.585471,53,39.68,0.748679,maintain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,88cd3681e0a2ba85e190f7d817629c4c,paid_search,b48ec5f3b04e9068441002a19df93c6c,237,reseller,online_big,car_accessories,SC,570.534070,37,160.64,4.341622,maintain
375,e116c7455dd26a4d8c3e92532583905f,unknown,b76ef37428e6799c421989521c0e5077,10,reseller,online_big,health_beauty,SP,6.359304,73,21.98,0.301096,leave
376,8fb67a334bacec338885b4e9e1708073,paid_search,40dec9f3d5259a3d2dbcdab2114fae47,3,manufacturer,online_medium,home_decor,PR,730.266597,79,21.38,0.270633,maintain
377,0cab2da43793a6f3c5ed8514c5f54627,NaN,6b6c0934205e87702519d1cce24055f9,24,reseller,offline,bags_backpacks,PR,540.532197,113,252.50,2.234513,maintain


In [37]:
order_data

,order_id,product_id,order_item_id,price,freight_value,seller_profit,distance,order_approved_at,seller_id
0,1efcc6ba75b519535a510c53153a9256,e94df75b2e9a8b056be0c144a731f55c,1,94.20,18.76,75.44,357.306376,2018-06-20 22:20:21,2c43fb513632d29b3b58df74816f1b06
1,d8d281c1a1398a1e835fdc4adfb81e2a,9e0873e9ac04b12f282b81af5daebd2e,1,664.00,38.41,625.59,422.590461,2018-07-24 10:31:11,2c43fb513632d29b3b58df74816f1b06
2,9dfd8df84d4f07cc8da0f2fa0fe96075,ba74c6b75d2ad7503175809688d5a03c,1,178.99,8.99,170.00,4.710751,2018-07-20 04:43:20,7d13fca15225358621be4086e1eb0964
3,931ca162115da049cd8d7872b3667163,e8f487b9cb03fdb7daf2505822570853,1,68.00,14.06,53.94,382.665308,2018-08-25 08:50:12,8a432f4e5b471f8da497d7dc517666e2
4,e8f4435c1db6d7bf9e191fd9bd6a6682,38f2d4b26820d63dc2675d6fbfafe1bb,1,32.89,12.89,20.00,247.327339,2018-06-20 19:58:06,4c1c7281388a33dd06daac44f9fadbd1
...,...,...,...,...,...,...,...,...,...
5025,f9b709a84c50f277996aa9f181738b73,ef7f13c7694be202c48017b71801815c,1,49.90,24.70,25.20,1919.286173,2018-06-21 11:58:50,f46490624488d3ff7ce78613913a7711
5026,4471fc5ecfbb3b7fec1044fc6e6f6851,1b34de23397f9ce6cd0f7afa27b60eac,1,38.50,22.99,15.51,2473.976692,2018-08-08 18:15:21,c70a353f02429c00775a46a75fb787da
5027,56aa3b6b2c30314ee80914fea9543573,059edbecfc72eba55fbd079a25c4ab6b,1,35.00,9.00,26.00,11.911855,2018-08-08 20:04:53,c70a353f02429c00775a46a75fb787da
5028,935cbf337f50d03fc7ecfb5a601dc7fc,1b34de23397f9ce6cd0f7afa27b60eac,1,38.50,19.18,19.32,2121.435422,2018-07-25 08:25:14,c70a353f02429c00775a46a75fb787da


In [39]:
seller_data.to_csv("/content/drive/MyDrive/olist/EDA_seller_data.csv", index = False)

In [38]:
order_data.to_csv("/content/drive/MyDrive/olist/EDA_order_data.csv", index = False)

In [ ]:
# seller_data 데이터 설명
"""
seller_id	: 판매자 번호 (아이디)
origin : Olist 유입 경로
landing_page_id : Olist 유입시 랜딩된 페이지
date_for_registration : 판매자 신청부터 등록까지 소요된 시간
business_type : 영업 방식 (중간 유통, 제작자 등)
lead_type : 영업 형태
business_segment : 취급 물품
seller_state : 판매 위치 (주)
mean_distance : 평균 판매 거리
sale_days : 등록 이후 영업 날짜
total_profit : 총 수익
LTV : 하루 평균 매출 기대값
seller_status : 판매자 이탈 여부 (최근 60 일간 판매가 없을 시 이탈로 간주)
"""

In [ ]:
# order_data 데이터 설명
"""
order_id : 주문 번호 (아이디)
product_id : 제품 번호 (아이디)
order_item_id	: 제품 구분 번호 (동일 제품 다수 구매시 중복 데이터로 처리 되는것을 방지)
price : 가격
freight_value : 운송료
seller_profit	: 판매자의 수익 = 가격 - 운송료
distance : 주문 배달 거리
order_approved_at : 결제 승인 시각
seller_id : 판매자 번호 (아이디)
"""